<h1> Approach </h1>
gaussian subsampling OR FFT preprocessing -> CNN + RNN https://github.com/anlthms/sp-2016

<h1> Notes </h1>
https://workflowy.com/s/sKHWm3chWc

<h1> Domain Tools </h1>
EEGrunt: A Collection Python EEG (+ ECG) Analysis Utilities for OpenBCI and Muse
https://github.com/curiositry/EEGrunt

Intro to MEG and EEG processing with MNE and Python
http://mne-tools.github.io/mne-python-intro/

MNE: MEG + EEG Analysis and visualization
MNE is a community-driven software package designed for processing electroencephalography (EEG) and magnetoencephalography (MEG) data
http://martinos.org/mne/stable/index.html

EEG Tools: Collection of Python modules for EEG analysis. Includes EDF+ and BDF loaders, scalp plots and commonly used spatial filters.
https://github.com/breuderink/eegtools

In [1]:
# translation of the Matlab feature extractor
from tqdm import tqdm
import sys
import os
import numpy as np
import pandas as pd
from math import *
from scipy.io import loadmat
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
%matplotlib inline
#import pyeeg
# pyeeg is the one that has very good fractal dimensions 
# computation but not installed here

In [2]:
try:
    st = pd.HDFStore("features_train1.h5")
    print(st.keys()[:4])
    kk = st.keys()[0]
    samplelist = []
    for kk in tqdm(st.keys()):
        label = int(kk.split('.')[0][-1])
        shortkey = kk[1:-4]
        sample = st[kk]
        sample["label"] = label
        sample["key"] = shortkey
        sample.index = sample.index.map(lambda x: shortkey + "_" + x)
        samplelist.append(sample)
#         break
finally:
    st.close()
    samplelist = pd.concat(samplelist)

['/1_1000_0.mat', '/1_1001_0.mat', '/1_1002_0.mat', '/1_1003_0.mat']


In [3]:
# samplelist.to_csv("manual_features.csv")

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_curve


from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [5]:
# train_data.iloc[:,:-2] = StandardScaler().fit_transform(train_data.iloc[:,:-2])
samplelist =  samplelist.dropna()
all_X = samplelist.iloc[:,:-2].as_matrix()
all_y = samplelist["label"].as_matrix()
train_data, val_data = train_test_split(samplelist.dropna(), test_size=0.2)

In [6]:
tr_X = train_data.iloc[:,:-2].as_matrix()
tr_y = train_data["label"].as_matrix()

v_X = val_data.iloc[:,:-2].as_matrix()
v_y = val_data["label"].as_matrix()

# GBMs !!!

In [7]:
from sklearn import cross_validation
from sklearn.ensemble import GradientBoostingClassifier
# num_instances 
num_trees = 10
num_folds = 5
seed = 5
num_instances = len(all_X)
kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)

model = GradientBoostingClassifier( n_estimators=num_trees, random_state=seed)
results = cross_validation.cross_val_score(model, all_X, all_y, cv=kfold)
print(results.mean())

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.84606244326


In [8]:
from sklearn import cross_validation
from sklearn.ensemble import GradientBoostingClassifier
# num_instances 
num_trees = 5
num_folds = 5
seed = 6
num_instances = len(all_X)
kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)

model = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed, 
                                   min_samples_leaf=3, max_depth=3,
                                  learning_rate=0.1, subsample=0.8, max_features=0.6)
results = cross_validation.cross_val_score(model, all_X, all_y, cv=kfold)
print(results.mean())

0.864546753503


In [ ]:
"to produce predictions do:"

model.fit(all_X, all_y)
# model.predict()



In [9]:
from sklearn import cross_validation
from sklearn.ensemble import GradientBoostingClassifier
# num_instances 
num_trees = 5
num_folds = 5
seed = 6
num_instances = len(all_X)
kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
model = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed, 
                                   min_samples_leaf=1, max_depth=3,
                                  learning_rate=0.1, subsample=0.8, max_features=0.6)
results = cross_validation.cross_val_score(model, all_X, all_y, cv=kfold)
print(results.mean())

0.865080086836
